In [2]:
from preprocessing.preprocessing import image_scaling, convert_image, label_conversion
from model.food_classifier import FoodDataset
import torch
import torchinfo
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
import os
import json
import gc
from PIL import Image
import numpy as np
import timm
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

ModuleNotFoundError: No module named 'preprocessing'

In [ ]:
image_folder = 'images'
data_folder = 'data'
BATCH_SIZE = 32
# Load calorie database (for food labels)
CALORIE_DB_FILE = os.path.join(data_folder, "calories_database.json")
with open(CALORIE_DB_FILE, "r") as f:
    calorie_db = json.load(f)

FOOD_LABELS = sorted(list(calorie_db.keys()))
NUM_CLASSES = len(FOOD_LABELS)

In [ ]:
resnet18 = timm.create_model('resnet18', pretrained=True, num_classes=NUM_CLASSES)
resnet26 = timm.create_model('resnet26', pretrained=True, num_classes=NUM_CLASSES)
resnet34 = timm.create_model('resnet34', pretrained=True, num_classes=NUM_CLASSES)
resnet50 = timm.create_model('resnet50', pretrained=True, num_classes=NUM_CLASSES)
# resnet101 = timm.create_model('resnet101', pretrained=True, num_classes=NUM_CLASSES)
# resnet200 = timm.create_model('resnet200')#, pretrained=True)
vittiny = timm.create_model('vit_tiny_patch16_384', pretrained=True, num_classes=NUM_CLASSES)
vitsmall16 = timm.create_model('vit_small_patch16_384', pretrained=True, num_classes=NUM_CLASSES)
vitsmall32 = timm.create_model('vit_small_patch32_384', pretrained=True, num_classes=NUM_CLASSES)
# vitlarge16 = timm.create_model('vit_large_patch16_384', pretrained=True, num_classes=NUM_CLASSES)
# vitlarge32 = timm.create_model('vit_large_patch32_384', pretrained=True, num_classes=NUM_CLASSES)
# vitbase16 = timm.create_model('vit_base_patch16_384', pretrained=True, num_classes=NUM_CLASSES)
# vitbase32 = timm.create_model('vit_base_patch32_384', pretrained=True, num_classes=NUM_CLASSES)

resnet_models = {
    'ResNet18': resnet18,
    'ResNet26': resnet26,
    'ResNet34': resnet34,
    'ResNet50': resnet50
    # 'ResNet101': resnet101,
    # 'ResNet200': resnet200
}

vit_models = {
    'ViT-Tiny': vittiny,
    'ViT-Small 16': vitsmall16,
    'ViT-Small 32': vitsmall32
    # 'ViT-Large 16' : vitlarge16,
    # 'ViT-Large 32' : vitlarge32,
    # 'ViT-Base 16': vitbase16,
    # 'ViT-Base 32': vitbase32
}

In [ ]:
# Define Data Transforms
transform = transforms.Compose([
    transforms.Resize((400, 400)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

transform384 = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load Datasets
train_dataset = FoodDataset(json_path=os.path.join(data_folder, "train.json"), 
                            img_dir="images_resized",
                            transform=transform)
test_dataset = FoodDataset(json_path=os.path.join(data_folder, "test.json"), 
                          img_dir="images_resized",
                          transform=transform)
train_dataset384 = FoodDataset(json_path=os.path.join(data_folder, "train.json"), 
                            img_dir="images_resized",
                            transform=transform384)
test_dataset384 = FoodDataset(json_path=os.path.join(data_folder, "test.json"), 
                          img_dir="images_resized",
                          transform=transform384)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
train_loader384 = DataLoader(train_dataset384, batch_size=BATCH_SIZE, shuffle=True)
test_loader384 = DataLoader(test_dataset384, batch_size=BATCH_SIZE, shuffle=False)

In [7]:
def train(model, trainloader, optimizer, criterion, device, epochs=10):
    for epoch in range(epochs):
        running_loss = 0
        model.train()
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device).float()
            optimizer.zero_grad()
            out = model(images)
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # delete from memory
            del images
            del labels
            torch.cuda.empty_cache()
            gc.collect()
            
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

In [8]:
def evaluate(model, testloader, device):
    model.eval() # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            unit_step_activation = torch.where(outputs > 0, torch.ones_like(outputs), torch.zeros_like(outputs))
            total += labels.numel()
            correct += (unit_step_activation == labels).sum().item()

            del images
            del labels
            torch.cuda.empty_cache()
            gc.collect()

    return 100 * correct / total

In [9]:
results = []

for rn_model in resnet_models.keys():
    model = resnet_models[rn_model].to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
    print(f"Training model {rn_model}")
    train(model, train_loader, optimizer, criterion, device)
    print("Training complete")
    accuracy = evaluate(model, test_loader, device)
    print(f"{rn_model} Test Accuracy: {accuracy}%")
    results.append({
        'model name': rn_model,
        'test accuracy': accuracy
    })
    del model
    torch.cuda.empty_cache()
    gc.collect()

for vit_model in vit_models.keys():
    model = vit_models[vit_model].to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
    print(f"Training model {vit_model}")
    train(model, train_loader384, optimizer, criterion, device)
    print("Training complete")
    accuracy = evaluate(model, test_loader384, device)
    print(f"{vit_model} Test Accuracy: {accuracy}%")
    results.append({
        'model name': vit_model,
        'test accuracy': accuracy
    })
    del model
    torch.cuda.empty_cache()
    gc.collect()

results_df = pd.DataFrame(results)
results_df

Training model ResNet18
Epoch 1/10, Loss: 4.943054974079132
Epoch 2/10, Loss: 3.160201515470232
Epoch 3/10, Loss: 2.448873622076852
Epoch 4/10, Loss: 2.277694340263094
Epoch 5/10, Loss: 2.1228376924991608
Epoch 6/10, Loss: 2.045820278780801
Epoch 7/10, Loss: 2.0365512328488484
Epoch 8/10, Loss: 1.9809543873582567
Epoch 9/10, Loss: 1.9895618898527963
Epoch 10/10, Loss: 1.9710689868245805
Training complete
ResNet18 Test Accuracy: 88.77911079745942%
Training model ResNet26
Epoch 1/10, Loss: 3.5011992880276273
Epoch 2/10, Loss: 2.5404027615274702
Epoch 3/10, Loss: 2.3086078422410146
Epoch 4/10, Loss: 2.2291954457759857
Epoch 5/10, Loss: 2.1681939448629106
Epoch 6/10, Loss: 2.0865977874823978
Epoch 7/10, Loss: 2.1430258623191287
Epoch 8/10, Loss: 2.116500062601907
Epoch 9/10, Loss: 2.028761182512556
Epoch 10/10, Loss: 2.039588817528316
Training complete
ResNet26 Test Accuracy: 90.0846859562456%
Training model ResNet34
Epoch 1/10, Loss: 5.191759475639889
Epoch 2/10, Loss: 3.3474735617637634


,model name,test accuracy
0,ResNet18,88.779111
1,ResNet26,90.084686
2,ResNet34,88.567396
3,ResNet50,92.448836
4,ViT-Tiny,65.702188
5,ViT-Small 16,69.442484
6,ViT-Small 32,65.314044
